### 환경설정

In [1]:
# 1. java 설치(version 1.7 이상)
# cmd 창에서 java --version으로 확인
# 2. 환경 변수 설정
# 제어판 > 시스템 > 고급 시스템 설정 > 고급 > 환경변수
# 제어판을 들어가지 않고 환경 변수를 확안하는 방법
import os
print('JAVA_HOME' in os.environ)
# True가 나온다면 자바 환경 변수까지 설정이 완료가 된 것!
# 만약 false뜬다면 별도의 환경변수 설정이 필요하다.
# -> jvm.dll이 있는 폴더를 시스템 변수에 JAVA_HOME으로 추가를 해주면 된다.
os.environ['JAVA_HOME'] = r'C:\Program Files\Java\jdk-11\bin\server'

True


In [2]:
print('JAVA_HOME' in os.environ)

True


In [3]:
# 환경 변수 설정 완료 후 Jpype1 설치
!pip install jpype1

In [4]:
# jpype1 설치 완료 후 한국어 형태소 분석기인 konlpy 설치하기
!pip install konlpy

In [5]:
#라이브러리 불러와보기
from konlpy.tag import Kkma

In [6]:
kkma = Kkma()

### 목표설정
- 네이버 영화리뷰 데이터 셋을 이용해서 긍정/부정 분류기를 만들어보자.
- TF-IDF 토큰화 방법을 사용해보자
- konlpy한국어 형태소 분석기를 사용해보자(글자 및)
- 단어별 긍/부정 정보를 시각화해서 확인해보자

In [7]:
#필요한 라이브러리 불러오기 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
df_train = pd.read_csv('./data/ratings_train.csv')
df_test = pd.read_csv('./data/ratings_test.csv')

In [9]:
pd.set_option('display.max_colwidth', None) # 컬럼에 폭 제한을 없애버리겠다!
pd.set_option('display.max_rows', None) # 전체 행을 보여줘라!

In [10]:
df_train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다,1


**컬럼 정리**
- id : 사용자 고유 식별 번호
- document : 실제 리뷰 내용
- label : 긍정/부정 -> 0: 부정리뷰, 1 : 긍정리뷰

In [11]:
#데이터 프레임 확인 - info()
# train의 정보 확인
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [12]:
# test의 정보 확인
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


- train 데이터 : 총 데이터 개수 : 15만개 / 결측치 5개
- test 데이터 : 총 데이터 개수 : 5만개 / 결측치 3개
- 데이터 볼륨에 비해 결측치의 양이 적다.
- 실제 리뷰 데이터를 채워주기엔 무리가 있다.
- 결측치가 있는 부분을 삭제하도록 하자.

In [13]:
# 결측치 데이터 삭제
# dropna -> 결측치가 있는 데이터를 삭제하는 함수
df_train.dropna(inplace = True)

In [14]:
df_test.dropna(inplace = True) 

In [15]:
# 데이터 확인
df_train.info()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


- 현재 데이터는 train, test로 나눠져있다.
- 우리가 머신러닝을 진행하기 위해서는 4가지의 데이터셋으로 나눠줄필요가 있다.
- 데이터를 분할해보자

In [16]:
# hint1. 텍스트 마이닝 실습을 진행하기 위해서는 리뷰 데이터가 필요하다
# hint2. 정답은 어떤 컬럼을 써야하는지 알아보자
# hint3. id 컬럼은 필요할까? 고민 한번 해보자


X_train = df_train['document']
y_train = df_train['label']

X_test = df_test['document']
y_test = df_test['label']
# 데이터의 형태를 우선적으로 파악하고, 어떻게 분할해야 할 지 생각해보자.

In [17]:
#데이터 분할 확인
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((149995,), (49997,), (149995,), (49997,))

**Vectorizer**

- Vectorizer는 말 그대로 어떠한 대상을 벡터화 시켜주는 도구다!
- (수치 값으로 바꿔주는 도구이다.)
- 기계학습 분야에서는 주로 텍스트를 쉽게 분석하기 위해서 벡터로 데이터를 표현할 때 사용
- 수치 값(숫자/벡터)으로 입력 값을 넣어주길 바라는 머신러닝 모델을 실행하기 위해서는 텍스트나 또 다른 데이터의 형태를 숫자나 벡터로 나타낼 필요가 있고 실현하기 위해 사용하는 것이 Vectorizer 이다.

### BOW(Bag Of Word) - 단어가방
- 문서를 벡터로 변환하는 가장 기본적인 방법으로 문서 내부에 모든 단어를 모아서 하나의 가방을 만들고, 특정 문서에 어떤 단어가 있는지 리스트의 형태로 순서 - 숫자형태로 나타내는 방법

In [18]:
# TF-IDF Vectorizer import
# 사이킷런에 있는 특성추출 도구 중에서 텍스트 추출 도구를 가져올거고
# 그중에서 TfidfVectorizerfmf rkwudhk wnj
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
X_train[:3]

0                  아 더빙.. 진짜 짜증나네요 목소리
1    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                    너무재밓었다그래서보는것을추천한다
Name: document, dtype: object

In [20]:
# TF-IDF 객체 생성
tf_idf_vect = TfidfVectorizer()
# 단어사전(단어 가방) 구축 - fit()
tf_idf_vect.fit(X_train[:3])
# 분리된 문장으로 만든 단어 사전 출력
tf_idf_vect.vocabulary_.keys()

dict_keys(['더빙', '진짜', '짜증나네요', '목소리', '포스터보고', '초딩영화줄', '오버연기조차', '가볍지', '않구나', '너무재밓었다그래서보는것을추천한다'])

- 언어의 특성을 따라 분할이 조금 달라지는 것 같다.
- 사이킷런은 외국에서 만들어졌기 때문에 외국의 언어체계를 따르는 것 같다.
- 영어나 다른 외국어 같은 경우는 단어 단위/ 띄어쓰기 단위로 분할을 해도 무리가 없다. -> 기본적으로 어절 단위로 끊어주기 때문에 한국어 특성상 파악이 어렵다.
- 조금 더 정확하게 벡터화를 진행하기 위해서 가장 작은 단위인 형태소 별로 나눠서 파악을 진행해보자!

### 한국어 형태소 분석기 konlpy 사용해보기

In [21]:
# 형태소 분석기 불러오기
from konlpy .tag import Kkma
# Hannanum, Kkma, komoran, mecab이라는 형태소 분석기가 존재
#여기서 mecab은 리눅스 환경에서만 작동하는 라이브러리 

**Kkma 라이브러리**
- Kkma : 꼬꼬마, 서울대학교 IDS(Intelligent Data System) 연구소 개발
- nouns() : 명사 추출
- morphs() : 형태소 추출
- pos() : 형태소 추출 + 품사 태그 부착

In [22]:
# 한국어 형태소 분석기 객체 생성
kkma = Kkma()

In [23]:
# 문장 하나 추출
X_train[0]

'아 더빙.. 진짜 짜증나네요 목소리'

In [24]:
# 명사 추출 : nouns()
kkma.nouns(X_train[0])

['더빙', '목소리']

In [25]:
# 우리가 사용해봤던 TF-IDF와 Kkma를 연결해서 사용해보자.
# step1. 함수 제작: 토큰화 작업을 해주는 함수를 제작해보자
def myTokenizer(text):
    return kkma.nouns(text)

In [26]:
# step 2. 기존 tf_ids에 위에서 제작한 함수 설정해주기
temp_tf_idf = TfidfVectorizer(tokenizer=myTokenizer)
# step 3. 단어사전 구축
temp_tf_idf.fit(X_train[0:3])
#step 4. 단어사전 확인
temp_tf_idf.vocabulary_.keys()

C:\Users\user04\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


dict_keys(['더빙', '목소리', '흠', '포스터', '포스터보고', '보고', '초', '초딩영화줄', '딩', '영화', '줄', '오버', '오버연기', '연기', '재', '재밓', '밓', '추천'])

In [27]:
temp_tf_idf.vocabulary_

{'더빙': 0,
 '목소리': 2,
 '흠': 17,
 '포스터': 15,
 '포스터보고': 16,
 '보고': 4,
 '초': 12,
 '초딩영화줄': 13,
 '딩': 1,
 '영화': 6,
 '줄': 11,
 '오버': 7,
 '오버연기': 8,
 '연기': 5,
 '재': 9,
 '재밓': 10,
 '밓': 3,
 '추천': 14}

### pos tagging 활용
- 형태소를 추출해서 품사 태깅까지 부착해주는 기능

In [28]:
# 품사 태그 종류 출력해보기
kkma.tagset

{'EC': '연결 어미',
 'ECD': '의존적 연결 어미',
 'ECE': '대등 연결 어미',
 'ECS': '보조적 연결 어미',
 'EF': '종결 어미',
 'EFA': '청유형 종결 어미',
 'EFI': '감탄형 종결 어미',
 'EFN': '평서형 종결 어미',
 'EFO': '명령형 종결 어미',
 'EFQ': '의문형 종결 어미',
 'EFR': '존칭형 종결 어미',
 'EP': '선어말 어미',
 'EPH': '존칭 선어말 어미',
 'EPP': '공손 선어말 어미',
 'EPT': '시제 선어말 어미',
 'ET': '전성 어미',
 'ETD': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JK': '조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKI': '호격 조사',
 'JKM': '부사격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JX': '보조사',
 'MA': '부사',
 'MAC': '접속 부사',
 'MAG': '일반 부사',
 'MD': '관형사',
 'MDN': '수 관형사',
 'MDT': '일반 관형사',
 'NN': '명사',
 'NNB': '일반 의존 명사',
 'NNG': '보통명사',
 'NNM': '단위 의존 명사',
 'NNP': '고유명사',
 'NP': '대명사',
 'NR': '수사',
 'OH': '한자',
 'OL': '외국어',
 'ON': '숫자',
 'SE': '줄임표',
 'SF': '마침표, 물음표, 느낌표',
 'SO': '붙임표(물결,숨김,빠짐)',
 'SP': '쉼표,가운뎃점,콜론,빗금',
 'SS': '따옴표,괄호표,줄표',
 'SW': '기타기호 (논리수학기호,화폐기호)',
 'UN': '명사추정범주',
 'VA': '형용사',
 'VC': '지정사',
 'VCN': "부정 지정사, 형용사 '아니다'",
 'VC

In [29]:
# 단순 형태소 추출 - morphs()
data = '먹다 먹는다 먹었다 아름답다 멋지다 안경 마스크 사진 모자'
kkma.morphs(data)

['먹',
 '다',
 '먹',
 '는',
 '다',
 '먹',
 '었',
 '다',
 '아름답',
 '다',
 '멋지',
 '다',
 '안경',
 '마스크',
 '사진',
 '모자']

In [30]:
# 우리가 원하는 품사만 골라와서 추출해보자
d = pd.DataFrame(kkma.pos(data), columns=['morphs', 'tag'])
d.set_index('tag', inplace = True)

In [31]:
# 텍스트 감성 분석에는 움직임을 나타내는 동사와, 상태를 표현하는 형용사가 유리하다.
# 기본적으로 많이 사용되는 보통 명사와 동사, 형용사만 뽑아와서 데이터 프레임 만들어보자
d.loc[d.index.intersection(['VV', 'VA', 'NNG'])]
#intersection() : 교집합 함수

,morphs
tag,
VV,먹
VV,먹
VV,먹
VA,아름답
VA,멋지
NNG,안경
NNG,마스크
NNG,사진
NNG,모자


In [34]:
# 명사, 형용사, 동사만 추출해서 데이터 프레임을 만들어보자
# 샘플데이터에 위의 방법으로 Tokenizer 적용해보기
def myTokenizer2(text):
    d=pd.DataFrame(kkma.pos(text),columns=['morphs', 'tag'])
    d.set_index('tag', inplace=True)
    if('VV' in d.index) | ('VA' in d.index) | ('NNG' in d.index):
        return d.loc[d.index.intersection(['VV', 'VA', 'NNG']), 'morphs'].values
    else :
        return[]

In [35]:
temp_tf_idf = TfidfVectorizer(tokenizer=myTokenizer2)
# 단어사전 만들기
temp_tf_idf.fit(X_train[:3])
# 단어사전 확인하기
temp_tf_idf.vocabulary_

C:\Users\user04\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


{'아': 5,
 '짜증나': 12,
 '더빙': 1,
 '목소리': 2,
 '흠': 15,
 '포스터': 14,
 '보고': 4,
 '영화': 8,
 '줄': 11,
 '오버': 9,
 '연기': 7,
 '가볍': 0,
 '재': 10,
 '추천': 13,
 '어': 6,
 '보': 3}

### 단어 사전을 구축해보자

In [36]:
final_tf_idf = TfidfVectorizer(tokenizer=myTokenizer2)
final_tf_idf.fit(X_train[:10000]) # 5분 정도 걸림

C:\Users\user04\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(tokenizer=<function myTokenizer2 at 0x00000144A86E2DD0>)

In [37]:
# 구축한 단어사전 갯수 확인
print('구축한 단어사전 갯수', len(final_tf_idf.vocabulary_))

구축한 단어사전 갯수 8748


In [38]:
# 단어사전을 이용해서 실제 단어를 벡터 값으로 변환
X_train = final_tf_idf.transform(X_train[:10000])
X_test = final_tf_idf.transform(X_test[0:10000])

In [39]:
# 정답 데이터도 갯수 맞춰주기
y_train = y_train[:10000]
y_test = y_test[:10000]

### 모델링
- 모델은 로지스틱 회귀를 사용
- 텍스트 마이닝은 시간이 오래 걸리는 분야이다. 상대적으로 속도가 빠른 로지스틱 회귀 모델이 선호된다.

In [40]:
# 모델 import
from sklearn.linear_model import LogisticRegression

In [41]:
# 모델 객체 생성
logi = LogisticRegression()

In [42]:
# 모델 학습
logi.fit(X_train, y_train)

LogisticRegression()

In [43]:
# 모델 정확도 평가
logi.score(X_test, y_test)

0.7843

In [44]:
# 학습시킨 모델을 활용해거 간단한 분류기 만들어보자(쌤이랑 같이 해볼거!)
target_name = ['부정', '긍정']
review = ['오랜만에 볼만한 한국영화였어요.']
vect_review = final_tf_idf.transform(review)
pre = logi.predict(vect_review)
print(f"'{review[0]}'는 {logi.predict_proba(vect_review).max()*100:2f}%로 {target_name[pre[0]]}  리뷰입니다.")

'오랜만에 볼만한 한국영화였어요.'는 81.493796%로 긍정  리뷰입니다.


In [46]:
#단어 사전 단어 순서대로 보기
final_tf_idf.vocabulary_

{'아': 4671,
 '짜증나': 7146,
 '더빙': 1742,
 '목소리': 2611,
 '흠': 8699,
 '포스터': 8014,
 '보고': 3208,
 '영화': 5271,
 '줄': 6884,
 '오버': 5359,
 '연기': 5172,
 '가볍': 55,
 '재': 6338,
 '추천': 7428,
 '어': 4938,
 '보': 3205,
 '교도소': 693,
 '이야기': 5907,
 '재미': 6353,
 '점': 6532,
 '조정': 6728,
 '멀': 2477,
 '없': 5041,
 '익살': 5974,
 '스파이': 4371,
 '더': 1730,
 '맨': 2450,
 '커스틴': 7565,
 '돋보이': 1829,
 '늙': 1489,
 '보이': 3238,
 '하': 8148,
 '이쁘': 5879,
 '걸음마': 353,
 '초등학교': 7354,
 '학년': 8209,
 '생인': 3875,
 '용': 5522,
 '별': 3186,
 '반개': 2939,
 '떼': 2070,
 '아깝': 4677,
 '원작': 5630,
 '긴장감': 1044,
 '살리': 3740,
 '욕': 5513,
 '이응': 5925,
 '경': 417,
 '길': 1047,
 '우': 5538,
 '생활': 3882,
 '발': 2969,
 '납': 1281,
 '감금': 145,
 '반복': 2947,
 '이': 5818,
 '드라마': 1947,
 '가족': 92,
 '사람': 3649,
 '모': 2550,
 '엿': 5233,
 '나오': 1205,
 '낫': 1284,
 '액션': 4881,
 '안': 4775,
 '있': 6141,
 '평점': 7994,
 '우드': 5541,
 '식': 4487,
 '화려': 8540,
 '낮': 1289,
 '헐': 8403,
 '길들이': 1048,
 '인피': 6036,
 '니트': 1507,
 '짱': 7158,
 '나': 1168,
 '죽': 6869,
 '때': 2042,
 '

In [49]:
# 단어 사전 정렬해서 데이터 프레임 만들기
voc = pd.DataFrame(final_tf_idf.vocabulary_.keys(),
                  index= final_tf_idf.vocabulary_.values())
voc.sort_index()

,0
0,ㄱ
1,ㄴ
2,ㄷ
3,ㄹ
4,ㅁ
5,ㅂ
6,ㅅ
7,ㅇ
8,ㅈ
9,ㅊ


In [50]:
# 가중치 확인
learning_result = pd.DataFrame(logi.coef_.T, index = voc.sort_index()[0], columns=['w'])
learning_result.sort_values(by='w')

,w
0,
재미없,-4.478074
최악,-4.288826
아깝,-4.098801
쓰레기,-4.057784
실망,-3.632922
없,-3.269283
유치,-2.753715
돈,-2.473668
노,-2.443241


- 하나의 문장이 들어갔을 때 벡터화 된 단어에 가중치 합을 구하게 된다.
- 마지막 가중치 합이 양수가 나오면 긍정적으로 분류
- 마지막 가중치 합이 음수가 나오면 긍정적으로 분류